# Importing Libraries

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import datasets, transforms, models

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score, precision_score, f1_score

# Defining Data Paths

In [2]:
data_dir = ""
train_dir = f"{data_dir}train"
val_dir = f"{data_dir}test"

# Data Prep

In [3]:
# Define the transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])
    
# Load the training dataset for mean and std calculation
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

# Function to calculate mean and std
def calculate_mean_std(loader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    
    for data, _ in loader:
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_squared_sum += torch.mean(data ** 2, dim=[0, 2, 3])
        num_batches += 1
    
    mean = channels_sum / num_batches
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5
    
    return mean, std

mean, std = calculate_mean_std(train_loader)
print(f'Mean: {mean}')
print(f'Std: {std}')

# Define transformations using calculated mean and std
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

Mean: tensor([0.4130, 0.4433, 0.5031])
Std: tensor([0.2405, 0.2253, 0.2144])


# Loading Data

In [4]:
# Load the datasets with ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(train_dir, data_transforms['train']),
    'val': datasets.ImageFolder(val_dir, data_transforms['val'])
}

# Create data loaders
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True),
    'val': DataLoader(image_datasets['val'], batch_size=32, shuffle=False)
}

# Get dataset sizes
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print(class_names)

['Bird-drop', 'Clean', 'Dusty', 'Electrical-damage', 'Physical-Damage', 'Snow-Covered']


# Defining Model

In [5]:
def set_device():
    if torch.cuda.is_available():
        dev = "cuda:0"
    else:
        dev = "cpu"
    return torch.device(dev)

device = set_device()
from torchvision.models import ResNet50_Weights
model = models.resnet50(weights=ResNet50Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 6)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training the Model

In [ ]:
num_epochs = 200
patience = 200
best_val_acc = 0.0  # Track the best validation accuracy
best_epoch = 0
early_stopping_counter = 0

for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')
    print('-' * 10)
    
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()
        
        running_loss = 0.0
        running_corrects = 0
        
        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        
        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        if phase == 'val':
            if epoch_acc > best_val_acc:
                best_val_acc = epoch_acc
                best_epoch = epoch
                early_stopping_counter = 0
                # Save the best model
                model_filename = f'best_solar_panel_model_{best_val_acc:.4f}.pth'
                torch.save(model.state_dict(), model_filename)
                print(f'Model saved: {model_filename}')
            else:
                early_stopping_counter += 1
    
    if early_stopping_counter >= patience:
        print(f'Early stopping at epoch {epoch}')
        break

print(f'Best Validation Accuracy: {best_val_acc:.4f} at epoch {best_epoch}')


Epoch 0/199
----------
train Loss: 1.5734 Acc: 0.3564
val Loss: 1.1293 Acc: 0.6265
Model saved: best_solar_panel_model_0.6265.pth
Epoch 1/199
----------
train Loss: 0.8015 Acc: 0.8052
val Loss: 0.7066 Acc: 0.8193
Model saved: best_solar_panel_model_0.8193.pth
Epoch 2/199
----------
train Loss: 0.4801 Acc: 0.8687
val Loss: 0.5742 Acc: 0.8494
Model saved: best_solar_panel_model_0.8494.pth
Epoch 3/199
----------
train Loss: 0.3032 Acc: 0.9351
val Loss: 0.4709 Acc: 0.8675
Model saved: best_solar_panel_model_0.8675.pth
Epoch 4/199
----------
train Loss: 0.2227 Acc: 0.9553
val Loss: 0.4409 Acc: 0.8735
Model saved: best_solar_panel_model_0.8735.pth
Epoch 5/199
----------
train Loss: 0.1412 Acc: 0.9827
val Loss: 0.4131 Acc: 0.8614
Epoch 6/199
----------
train Loss: 0.1139 Acc: 0.9827
val Loss: 0.3967 Acc: 0.8795
Model saved: best_solar_panel_model_0.8795.pth
Epoch 7/199
----------
train Loss: 0.0853 Acc: 0.9971
val Loss: 0.4104 Acc: 0.8675
Epoch 8/199
----------
train Loss: 0.0753 Acc: 0.9986
